# EDA & Cleaning of Additional Data

Notebook for obtaining and cleaning additional data to append to the original kaggle dataset.

## Contents

- [Oil Prices](#Oil-Prices)
- [Weather Forecasts](#Weather-Forecasts)
- [Save to csv](#Save-to-csv)

### Imports

In [5]:
# General Imports
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import chart_studio.plotly as py 
import plotly.graph_objs as go 
import plotly.express as px

# Webscraping Imports
import requests
from bs4 import BeautifulSoup
import time

### Oil Prices

As part of predicting electricity prices it is important to understand the underlying drivers. One of the primary drivers is [oil price](https://www.businessjuice.co.uk/energy-guides/what-drives-the-price-of-electricity/) which is also related to natural gas prices. 

We will gather historical data for:
- [Crude Oil Prices](https://www.exchangerates.org.uk/commodities/OIL-EUR-history.html)

Unfortunately we were only able to find this data daily and not hourly, however, it will hopefully still be useful for our analysis by attaching the daily price to the corresponding periods for each day.

**Crude Oil Webscraping**

We will gather historical crude oil prices in euros/barrel by scraping the [exchangerates.org.uk](https://www.exchangerates.org.uk/commodities/OIL-EUR-history.html) website.

In [2]:
# Set up for scraping
url = 'https://www.exchangerates.org.uk/commodities/OIL-EUR-history.html'
res = requests.get(url)
res.status_code

200

In [4]:
# Set up beautiful soup
soup = BeautifulSoup(res.content,'lxml')

In [7]:
# Scrape for prices and dates
# Adapted from GA DSI lesson 5.02
# 1. Find the container
table = soup.find('table', {'id': 'hist'})
# 2. Create the list
crude_oil = []
# 3. Loop through the rows
for row in table.find_all('tr')[2:]: # skips the 2 header rows
    # 4. Create the dictionary
    oil = {}
    # 5. Fill in the dictionary
    oil['date'] = row.find_all('td')[0].text.strip()
    oil['oil_price'] = row.find_all('td')[2].text.strip()
    # 6. Append the data for that date to the list of all dates
    crude_oil.append(oil)
# 7. Check dictionary
crude_oil[0:10]

[{'date': 'Wednesday  4 March 2020', 'oil_price': '42.49694'},
 {'date': 'Tuesday  3 March 2020', 'oil_price': '42.08195'},
 {'date': 'Monday  2 March 2020', 'oil_price': '43.00713'},
 {'date': 'Sunday  1 March 2020', 'oil_price': '40.37255'},
 {'date': 'Saturday 29 February 2020', 'oil_price': '41.06431'},
 {'date': 'Friday 28 February 2020', 'oil_price': '41.06441'},
 {'date': 'Thursday 27 February 2020', 'oil_price': '42.55531'},
 {'date': 'Wednesday 26 February 2020', 'oil_price': '44.49947'},
 {'date': 'Tuesday 25 February 2020', 'oil_price': '46.12790'},
 {'date': 'Monday 24 February 2020', 'oil_price': '47.40306'}]

In [9]:
len(crude_oil)/365

9.301369863013699

We have over 9 years of data going backwards from today, this completely covers the period of our initial data so this should be good. Let's convert to a dataframe and run some EDA. 

In [10]:
oil_prices = pd.DataFrame(crude_oil)
oil_prices.head()

,date,oil_price
0,Wednesday 4 March 2020,42.49694
1,Tuesday 3 March 2020,42.08195
2,Monday 2 March 2020,43.00713
3,Sunday 1 March 2020,40.37255
4,Saturday 29 February 2020,41.06431


In [11]:
oil_prices.shape

(3395, 2)

In [12]:
oil_prices.dtypes

date         object
oil_price    object
dtype: object

We need to convert date to a datetime object and oil_price to a float.

In [13]:
# Convert date to datetime
oil_prices['date'] = pd.to_datetime(oil_prices['date'])

In [17]:
# Make it the index
oil_prices.set_index('date', inplace=True)

In [18]:
oil_prices.head(2)

,oil_price
date,
2020-03-04,42.49694
2020-03-03,42.08195


In [33]:
oil_prices.index

DatetimeIndex(['2020-03-04', '2020-03-03', '2020-03-02', '2020-03-01',
               '2020-02-29', '2020-02-28', '2020-02-27', '2020-02-26',
               '2020-02-25', '2020-02-24',
               ...
               '2010-11-22', '2010-11-21', '2010-11-20', '2010-11-19',
               '2010-11-18', '2010-11-17', '2010-11-16', '2010-11-15',
               '2010-11-14', '1970-01-01'],
              dtype='datetime64[ns]', name='date', length=3395, freq=None)

We can see that there is one data point from back in 1970, we should get rid of that. We will keep the other data outside of our 2014 - 2018 range in case we would like to build on this further in the future.

In [35]:
oil_prices.drop(pd.to_datetime('1970-01-01'), inplace=True)

In [20]:
# Convert prices to float
oil_prices['oil_price'] = oil_prices['oil_price'].astype(float)

In [21]:
oil_prices.dtypes

oil_price    float64
dtype: object

We were succesful changing the datatypes. Now let's take a quick look at the data.

In [22]:
oil_prices.describe()

,oil_price
count,3395.000000
mean,57.623533
std,13.851133
min,0.000000
25%,45.894300
50%,55.578300
75%,71.116850
max,83.654600


Having a min of 0 seems very suspicious, let's take a closer look.

In [23]:
# Check 0 values
oil_prices[oil_prices['oil_price']==0]

,oil_price
date,
2017-09-25,0.0


Looking back at the table the price the day before was 42.53 euros and the price the next day was 43.96 euros. The 0 euro value is clearly and error and only appears once so let's fill it with an average of the previous day and the next day.

In [30]:
# Fix zero value
oil_prices.loc[oil_prices['oil_price']==0,'oil_price'] = (42.53+43.96)/2

In [31]:
oil_prices.describe()

,oil_price
count,3395.000000
mean,57.636271
std,13.817970
min,24.119100
25%,45.894300
50%,55.578300
75%,71.116850
max,83.654600


We have gotten rid of the 0 value, now let's plot the data

In [36]:
# adapted from Noelle Brown

# Adding multiple lines
fig = go.Figure()
fig.add_trace(go.Scatter(x=oil_prices.index, y=oil_prices['oil_price'], name='Crude Oil Price (euros)'))
fig.update_layout(title_text='Daily Crude Oil Price in Euros',
                  xaxis_rangeslider_visible=True)

fig.show()

There does not appear any errors in the chart and when we compare to a chart of [monthly prices] we can see that the data seems to follow the same general trend, so we can be confident in it's correctness.

### Weather Forecasts

While the initial dataset from kaggle has a large amount of weather data, the data is all current as of the actual hour. This makes it good for learning about how weather effects current prices, however, for forecasting future prices we want to know what the relevant weather forecasts are. 

We will use the [DarkSky API](https://darksky.net/) to get historical weather forecasts for Madrid and Barcelona and see if they can help our models. 

**Due to time constraints, fully integrating weather into our forecasting will happen at a future point.**

### Save to csv

In [40]:
oil_prices.to_csv('../Data/intermediary/oil_daily.csv')